In [56]:
import json

In [ ]:
dataset_path = "/HabitatLLM/data/datasets/partnr_episodes/v0_0/train_2k.json"
# captions_path = "/HabitatLLM/data/hssd-hab/metadata/filtered_captions.json"
captions_path = "/HabitatLLM/data/hssd-hab/metadata/object_captions_handle.json"

In [64]:
with open(dataset_path, "r") as f:
    dataset = json.load(f)

In [65]:
with open(captions_path, "r") as f:
    captions = json.load(f)

In [71]:
df = pd.read_csv(category_cluster)

In [66]:
episodes = dataset['episodes']

In [ ]:
import pandas as pd
category_cluster_path = "/HabitatLLM/data/hssd-hab/metadata/category_cluster.csv"
df = pd.read_csv(category_cluster_path)
mapper_path = "/HabitatLLM/data/hssd-hab/metadata/object_categories_filtered.csv"
mapper_df = pd.read_csv(mapper_path)

In [29]:
mapper_df.head()

id   clean_category
0                                         B00TS18AEA  battery_charger
1  Eat_to_Live_The_Amazing_NutrientRich_Program_f...             book
2                                            Book_23             book
3           c9d140d976657e8d058bd6bc3dc068b87daf9260             book
4                                         B078GTTCCF             book

In [30]:
from tqdm import tqdm

evals = []
for i, episode in tqdm(enumerate(episodes)):
    for eval in episode['evaluation_propositions']:
        func = eval['function_name']
        evals.append(func)

1000it [00:00, 214861.12it/s]


In [31]:
list(set(evals))

['is_filled',
 'is_powered_on',
 'is_powered_off',
 'is_next_to',
 'is_clean',
 'is_in_room',
 'is_inside',
 'is_on_floor',
 'is_on_top']

In [ ]:
from tqdm import tqdm

# is inside
# is on top 
# is in room
# is next to

filtered_episodes = []
for i, episode in tqdm(enumerate(episodes)):
    filter = False
    for eval in episode['evaluation_propositions']:
        func = eval['function_name']
        
        if func in ["is_on_top", "is_inside", "is_in_room"]:
            for obj in eval['args']['object_handles']:
                obj_name = obj.split(':')[0][:-1]
                caption = captions.get(obj_name)
            
                if caption is None:
                    filter = True
                    break
            
        elif func == "is_next_to":
            obj_a = eval['args']['entity_handles_a']
            obj_b = eval['args']['entity_handles_b']
            
            for obj in obj_a:
                obj_name = obj.split(':')[0][:-1]
                caption = captions.get(obj_name)
                map_res = mapper_df[mapper_df['id'] == obj_name]
            
                if caption is None:
                    if len(map_res)==0:
                        continue
                    else:
                        filter = True
                        break
            
            for obj in obj_b:
                obj_name = obj.split(':')[0][:-1]
                caption = captions.get(obj_name)
            
                map_res = mapper_df[mapper_df['id'] == obj_name]
            
                if caption is None:
                    if len(map_res)==0:
                        continue
                    else:
                        filter = True
                        break
        
        else:
            filter=True
    
    if not filter:
        filtered_episodes.append(episode)

2000it [00:01, 1765.46it/s]


In [68]:
len(filtered_episodes)

1401

In [23]:
final_dict = {
    "config": None,
    "episodes": filtered_episodes
}
with open("filtered_val_episodes.json", "w") as f:
    json.dump(final_dict, f, indent=4)

In [ ]:
from tqdm import tqdm

# is inside
# is on top 
# is in room
# is next to

filtered_episodes = []
for i, episode in tqdm(enumerate(episodes)):
    filter = False
    for eval in episode['evaluation_propositions']:
        func = eval['function_name']
        
        if func in ["is_on_top", "is_inside", "is_in_room"]:
            for obj in eval['args']['object_handles']:
                obj_name = obj.split(':')[0][:-1]
                caption = captions.get(obj_name)
                map_res = mapper_df[mapper_df['id'] == obj_name]

                if caption is None:
                    if len(map_res)==0: # receptacle
                        continue
                    else: # not receptacle
                        filtered_df = df[df['category'] == map_res['clean_category'].values[0]]
                        random_rows = filtered_df.groupby('cluster').sample(n=1, random_state=42)  # 각 cluster에서 1개씩 샘플링
                        random_rows = random_rows.sample(n=min(2, len(random_rows)), random_state=42) # 최대 2개만 선택 (cluster 개수가 2개 이상일 경우)

                        print(random_rows)
                        filter = True
                        break
                else:
                    obj_row = df[df['id'] == obj_name]
                    print(obj_row)
                    obj_cluster = obj_row['cluster'].values[0]  # obj_name의 cluster 값
                    obj_category = obj_row['category'].values[0]
                    
                    filtered_df = df[(df['category'] == obj_category) & (df['cluster'] != obj_cluster)]

                    # 3. 랜덤하게 1개 샘플링 (해당 데이터가 있을 경우에만)
                    if not filtered_df.empty:
                        random_row = filtered_df.sample(n=1, random_state=42)  # 같은 category이지만 다른 cluster 중 랜덤 샘플링
                        print(random_row)

                        filter = True
                        break
            
        elif func == "is_next_to":
            obj_a = eval['args']['entity_handles_a']
            obj_b = eval['args']['entity_handles_b']
            
            for obj in obj_a:
                obj_name = obj.split(':')[0][:-1]
                caption = captions.get(obj_name)
                map_res = mapper_df[mapper_df['id'] == obj_name]
            
                if caption is None:
                    if len(map_res)==0:
                        continue
                    else:
                        filter = True
                        break
            
            for obj in obj_b:
                obj_name = obj.split(':')[0][:-1]
                caption = captions.get(obj_name)
            
                map_res = mapper_df[mapper_df['id'] == obj_name]
            
                if caption is None:
                    if len(map_res)==0:
                        continue
                    else:
                        filter = True
                        break
        
        else:
            filter=True
    
    if not filter:
        filtered_episodes.append(episode)

0it [00:00, ?it/s]


IndexError: index 0 is out of bounds for axis 0 with size 0

In [57]:
from tqdm import tqdm

# is inside
# is on top 
# is in room
# is next to

filtered_episodes = []
for i, episode in tqdm(enumerate(episodes)):
    filter = False
    evals = []
    try:
        for eval in episode['evaluation_propositions']:
            func = eval['function_name']
            print(func)
            if func == "is_on_top" or "is_inside":
                obj = eval['args']['object_handles'][0].split(':')[0][:-1]
                recep = eval['args']['receptacle_handles'][0].split('_')[0]
                recep_key = mapper_df[mapper_df['id'] == recep]['wnsynsetkey'].values[0]
                
                caption = captions.get(obj)
                if caption is None:
                    filter = True
                    print(f"{obj} is None")
                    break
                eval_str = f"{obj}/{caption['category']} - {func} - {recep}/{recep_key}"
                evals.append(eval_str)
                
            elif func == "is_next_to":
                obj_a = eval['args']['entry_handles_a']
                obj_b = eval['args']['entry_handles_b']
                
                caption_a = captions.get(obj_a)
                caption_b = captions.get(obj_b)
                
                if caption_a is None or caption_b is None:
                    filter = True
                    print(f"{obj} is None")
                    break
                eval_str = f"{obj_a}/{caption_a['category']} - {func} - {obj_b}/{caption_b['category']}"
                evals.append(eval_str)
                
            elif func == "is_in_room":
                obj = eval['args']['object_handles'][0].split(':')[0][:-1]
                room = eval['args']['room_ids'][0]
            
                aption = captions.get(obj)
                
                if caption is None:
                    filter = True
                    print(f"{obj} is None")
                    break
                eval_str = f"{obj}/{caption['category']} - {func} - {room}"
                evals.append(eval_str)

        if not filter:
            filtered_episodes.append(episode)
            for e in evals:
                print(e)
    except Exception as e:
        print(e)
            
    print("==============================")

53it [00:00, 258.88it/s]

is_on_top
is_on_top
is_on_top
9dfbc1df2349093e0444dfabb8789296c83561cf/candle - is_on_top - 0239c03b6896e4e4fad328885cf078995c913046/coffee_table.n.01
B075HR7LD2/candle_holder - is_on_top - 0239c03b6896e4e4fad328885cf078995c913046/coffee_table.n.01
Ecoforms_Plant_Pot_GP9AAvocado/plant_container - is_on_top - 0239c03b6896e4e4fad328885cf078995c913046/coffee_table.n.01
is_on_top
is_on_top
8fba7557cf88cc31a6b44a28459b874dc494fe55/cushion - is_on_top - 8ac28a1a29cb5d49f55dd27dbd06d7f3749db356/easy_chair.n.01
CREATIVE_BLOCKS_35_MM/toy_construction_set - is_on_top - 0239c03b6896e4e4fad328885cf078995c913046/coffee_table.n.01
is_on_top
is_on_top
Fisher_price_Classic_Toys_Buzzy_Bee/toy_bee - is_on_top - 8ac28a1a29cb5d49f55dd27dbd06d7f3749db356/easy_chair.n.01
FIRE_ENGINE/toy_fire_truck - is_on_top - 8ac28a1a29cb5d49f55dd27dbd06d7f3749db356/easy_chair.n.01
is_on_top
is_on_top
is_on_top
is_on_top
Ecoforms_Plant_Saucer_S14MOCHA/plant_saucer - is_on_top - 689340fdf94601c7e57aabc45c1f1752bc7791da/end

80it [00:00, 259.37it/s]

577ab6846773c22e68a1ead745bba0ac6e60d8b2/electric_kettle - is_on_top - c39181c455b8df6ac1b8112a8d2d7bc4a493d8e7/console_table.n.01
6d3e6501fd91b94bedb34344757d43b8c5f8facf/kettle - is_on_top - c39181c455b8df6ac1b8112a8d2d7bc4a493d8e7/console_table.n.01
is_on_top
is_on_top
is_on_top
Thomas_Friends_Woodan_Railway_Henry/toy_vehicle - is_on_top - 44f35b8477fdbc61ed4c0ec40297b03d636d19b2/end_table.n.01
CREATIVE_BLOCKS_35_MM/toy_construction_set - is_on_top - 44f35b8477fdbc61ed4c0ec40297b03d636d19b2/end_table.n.01
Nintendo_Mario_Action_Figure/action_figure - is_on_top - 44f35b8477fdbc61ed4c0ec40297b03d636d19b2/end_table.n.01
is_on_top
is_on_top
Spatula_1/spatula - is_on_top - de47820f2dc539102408f43243369f2a952b7b2f/kitchen_island.n.01
Watch_1/watch - is_on_top - de47820f2dc539102408f43243369f2a952b7b2f/kitchen_island.n.01
is_on_top
is_on_top
is_on_top
Schleich_S_Bayala_Unicorn_70432/toy_animal - is_on_top - 31653c084558e1ecb286adf63a305d914aa3c98b/nightstand.n.01
Thomas_Friends_Wooden_Railw

139it [00:00, 273.19it/s]

...

is_on_top
PINEAPPLE_MARACA_6_PCSSET/toy_pineapple - is_on_top - 5e72623c1e3999e0fab6656621f8f6b23879a5a1/king_bed.n.01
PINEAPPLE_MARACA_6_PCSSET/toy_pineapple - is_on_top - c0b50a7f924f4738899cebd17412dc364bcfaf3e/bench.n.01
is_on_top
is_on_top
is_on_top
is_on_top
is_on_top
is_on_top
is_next_to
'object_handles'
is_on_top
is_on_top
is_on_top
is_on_top
is_on_top
is_on_top
is_next_to
'object_handles'
is_on_top
is_on_top
is_next_to
'object_handles'
is_on_top
is_on_top
is_next_to
'object_handles'
is_on_top
d494fa9ee269f2a557a274d9156ec60795051b80 is None
is_on_top
is_on_top
is_on_top
is_on_top
is_next_to
'object_handles'
is_on_top
is_on_top
is_on_top
is_on_top
is_on_top
is_on_top
is_next_to
'object_handles'
is_on_top
is_on_top
bb4b72c2ccb0a094041f26774be1930dd3e76855 is None
is_on_top
is_on_top
is_on_top
is_on_top
is_on_top
is_on_top
is_next_to
'object_handles'
is_on_top
is_on_top
is_on_top
is_on_top
is_on_top
is_on_top
Fertility_Statue_3/statue - is_on_top - 925f48efff312862677c9517a8ec0fa

833it [00:02, 347.91it/s]

is_on_top
is_next_to
'object_handles'
is_on_top
is_on_top
is_on_top
is_on_top
is_on_top
is_on_top
is_next_to
'object_handles'
is_on_top
is_on_top
is_on_top
is_on_top
is_on_top
is_on_top
5bb85d3caf11acb1eb2f299ccc724eb7a855b7cf/book - is_on_top - 6362fc0c5cc9d89330af89a9c19783a1ea2b0ee8/bed.n.01
f03a9160a43f06d49d579eb58f6ed63740afa2e4/clock - is_on_top - 6362fc0c5cc9d89330af89a9c19783a1ea2b0ee8/bed.n.01
74f64d7acc653e121e48e7f98dfdeba8b7f26067/stuffed_toy - is_on_top - 6362fc0c5cc9d89330af89a9c19783a1ea2b0ee8/bed.n.01
5bb85d3caf11acb1eb2f299ccc724eb7a855b7cf/book - is_on_top - 88cc870eaace923c89c47a1c3ffd3e48afdea16f/coffee_table.n.01
f03a9160a43f06d49d579eb58f6ed63740afa2e4/clock - is_on_top - 88cc870eaace923c89c47a1c3ffd3e48afdea16f/coffee_table.n.01
74f64d7acc653e121e48e7f98dfdeba8b7f26067/stuffed_toy - is_on_top - 88cc870eaace923c89c47a1c3ffd3e48afdea16f/coffee_table.n.01
is_on_top
fa86c98cbdba8962a436be647969fa0f304d2c84 is None
is_in_room
'receptacle_handles'
is_on_top
is_on_top


1000it [00:03, 324.73it/s]

is_on_top
is_clean
'receptacle_handles'
is_on_top
is_on_top
is_clean
'receptacle_handles'
is_on_top
is_on_top
is_filled
'receptacle_handles'
is_on_top
is_on_top
a8e2f442d0f6e5d86d27aa23aac71392f332addf is None
is_on_top
is_on_top
is_clean
'receptacle_handles'
is_on_top
is_on_top
is_powered_on
'receptacle_handles'
is_in_room
'receptacle_handles'
is_on_top
is_powered_on
'receptacle_handles'
is_in_room
'receptacle_handles'
is_in_room
'receptacle_handles'
is_on_top
is_on_top
is_clean
'receptacle_handles'
is_on_top
is_on_top
is_clean
'receptacle_handles'
is_on_top
is_on_top
0aebf7979e3510654f3c16418145ab973c22a381 is None
is_on_top
is_on_top
is_clean
'receptacle_handles'
is_on_top
is_on_top
is_clean
'receptacle_handles'
is_on_top
is_on_top
is_clean
'receptacle_handles'
is_on_top
edb1c6cfdba7c7a7f48e18db9194eb9efe0356a6 is None
is_on_top
is_on_top
is_clean
'receptacle_handles'
is_on_top
is_on_top
is_clean
'receptacle_handles'
is_on_top
is_on_top
is_filled
'receptacle_handles'
is_in_room
'rec

In [58]:
filtered_episodes = []
for i, episode in tqdm(enumerate(episodes)):
    filter = False
    evals = []
    try:
        for eval in episode['evaluation_propositions']:
            if func in ["is_on_top", "is_inside", "is_next_to", "is_in_room"]:
                if func == "is_next_to":
                    obj_a = eval['args']['entry_handles_a']
                    obj_b = eval['args']['entry_handles_b']

                    caption_a = captions[obj_a]
                    caption_b = captions[obj_b]
                    if caption_a is None or caption_b is None:
                        filter = True
                        print(f"{obj} is None")
                        break
                else:
                    obj = eval['args']['object_handles'][0].split(':')[0][:-1]
                    caption = captions.get(obj)
                
                    if caption is None:
                        filter = True
                        print(f"{obj} is None")

        if not filter:
            filtered_episodes.append(episode)
            for e in evals:
                print(e)
    except Exception as e:
        print(e)
            
    print("==============================")

1000it [00:00, 69599.99it/s]

'object_handles'
514bcfa03c15eae74fc4ee090c9e3186100ecede is None
c7a409e41936a6b12e017b30d9328aac5bf0de93 is None
f737d0e07f2927d7b4f43abb3a7267adfa5e3cf9 is None
a8e2f442d0f6e5d86d27aa23aac71392f332addf is None
edb1c6cfdba7c7a7f48e18db9194eb9efe0356a6 is None
8e2cdcd475c5b8676779bf6a24605861517818bd is None
7e067cafd8b96fae4671c0032317763fd9549b80 is None
5a0ac54dc91ce3d40e4351a0dcfb6b9ffa08fdc2 is None
050514a8a7fea2363cd924b7085dfc4207e6cbfd is None
xxxx9a19f571x1dc0x45c3x99fdxbf82a1abcb7f is None
xxxx041a75a1x3270x4573x8991xd08d08e3ae14 is None
'object_handles'
'object_handles'
'object_handles'
'object_handles'
'object_handles'
'object_handles'
'object_handles'
42b700c2f764ffffd3bba8d4851c12ef064c86d1 is None
'object_handles'
'object_handles'
'object_handles'
'object_handles'
'object_handles'
'object_handles'
'object_handles'
'object_handles'
4df6a26c581ab5ed73699c38c45f990270b75533 is None
'object_handles'
'object_handles'
'object_handles'
edb1c6cfdba7c7a7f48e18db9194eb9efe0356a6

In [59]:
print(len(filtered_episodes))

339
